In [1]:
import numpy as np
from util import hierarchical_cluster, construct_dendrogram, add_clusters_to_nodes
from helper import import_predictions, extract_features
import json

In [3]:
# IMPORT INSTANCE DATA
relative_folder = "./mnist_model"
predictions_filename = "vae_mnist_train.json"
mnist_data = import_predictions(f"{relative_folder}/{predictions_filename}")

In [4]:
# Extract information
dataset = mnist_data["dataset"]
model = mnist_data["model_name"]
instances = mnist_data["train_instances"]

print(mnist_data.keys())
print(model, dataset, sep=", ")
print(f"{len(instances)} instances")

dict_keys(['dataset', 'train_instances', 'model_name'])
conv_vae, mnist
60000 instances


In [6]:
features = extract_features(instances)
print(instances[0].keys(), features.shape)

dict_keys(['index', 'filename', 'features']) (60000, 20)


In [7]:
clusters = hierarchical_cluster(features, method="ward")

In [8]:
dendrogram = construct_dendrogram(clusters, connect_parent=True)
add_clusters_to_nodes(dendrogram)
# now I want to add counts
def add_node_count(node):
	node["node_count"] = len(node["cluster"])
dendrogram.for_each_node(add_node_count)

In [9]:
dendrogram.root["node_count"]

60000

In [10]:
# now I want to add extra information on the leaves (correspond to an image)
def add_info_to_leaves(dendrogram, instances, callback):
    for leaf in dendrogram.leaves:
        id = leaf["instance_index"]
        instance = instances[id]

        callback(leaf, instance) 

def add_filename(leaf, instance):
    file_key = "filename"

    filename = instance[file_key]
    leaf[file_key] = filename

add_info_to_leaves(dendrogram, instances, add_filename)

In [15]:
dendrogram.remove_attr("parent", "cluster")

In [12]:
dendrogram.root.keys()
dendrogram.leaves[0].keys()

dict_keys(['leaf', 'parent', 'merging_distance', 'node_index', 'instance_index', 'cluster', 'node_count', 'filename'])

In [16]:
with open("../mnist/clusters/vae_mnist_clusters.json", "w") as out:
    json.dump({"tree": dendrogram.dict_format}, out)